##### The following template demonstrates how to:
##### 1. Upload a document
##### 2. Retrieve the document
##### 3. Workflow management
##### 4. Form operation 

In [2]:
from pprint import pprint

TODO: Add the correct path below

In [3]:
DOCUMENT_PATH = "../AAPL_10Q.pdf"

## Uploading document

In [4]:
from weavaidev import documents as document_operation

print(f"Uploading document {DOCUMENT_PATH.split('/')[-1]}")

document_create_response = document_operation.create_document(file_path=DOCUMENT_PATH)

print("Uploaded document!")

/Users/adityakotwal/Desktop/WEAV/weav-ai-dev/package/.env
/Users/adityakotwal/Desktop/WEAV/weav-ai-dev/package/.env
/Users/adityakotwal/Desktop/WEAV/weav-ai-dev/package/.env
/Users/adityakotwal/Desktop/WEAV/weav-ai-dev/package/.env
/Users/adityakotwal/Desktop/WEAV/weav-ai-dev/package/.env
/Users/adityakotwal/Desktop/WEAV/weav-ai-dev/package/.env
Uploading document AAPL_10Q.pdf
Uploaded document!


In [5]:
document_id = document_create_response.id
print(f"Document ID: {document_id}")

Document ID: 6711d5a67212c1ec5f894923


## Get information about the document

In [6]:
get_document_response = document_operation.get_document(document_id=document_id)

print("Fetched document")

pprint(get_document_response.model_dump())

document_category = get_document_response.category

print(document_category)

Fetched document
{'ai_tags': [],
 'category': '',
 'created_at': datetime.datetime(2024, 10, 18, 3, 27, 34, tzinfo=TzInfo(UTC)),
 'download_url': '/doc-proc-service/local_store/google-oauth2|117349365869611297391/6711d5a67212c1ec5f894923/6711d5a67212c1ec5f894923',
 'file_name': 'AAPL_10Q.pdf',
 'form_instances': None,
 'id': '6711d5a67212c1ec5f894923',
 'in_folders': [],
 'media_type': 'application/pdf',
 'pages': [],
 'redacted_summary': '',
 'size': 654929,
 'source': 'application',
 'status': 'NEW',
 'step_status': {'FORM_EXTRACTION': {'error': '',
                                     'modified_at': datetime.datetime(2024, 10, 18, 3, 27, 34, tzinfo=TzInfo(UTC)),
                                     'response': {},
                                     'status': 'NOT_STARTED'}},
 'summary': '',
 'summary_status': '',
 'tags': [],
 'tenant_id': '',
 'user_id': 'google-oauth2|117349365869611297391'}



## Workflow management

### 1. Getting all workflows

In [7]:
from weavaidev import workflows as workflow_operation

print("GETTING ALL WORKFLOWS")

workflow_response = workflow_operation.get_all_workflows()

workflows = [workflow.name for workflow in workflow_response.workflows]

pprint(workflows)

GETTING ALL WORKFLOWS
['dagtasktest',
 'dagtest',
 'download_from_connector',
 'extract_form_and_compliance',
 'process_brain_document',
 'process_compliance_exclude_entity',
 'process_custom_queries',
 'process_document',
 'process_document_create_decision_tree',
 'process_document_multi_forms',
 'process_document_sensors',
 'process_document_summary_compliance',
 'process_document_summary_generation',
 'process_document_tabular_data',
 'process_form_data_cleanup',
 'process_form_workflow',
 'run_agent']


### 2. Running specific workflows

#### Process document workflow

In [8]:
print("RUNNING 'process_document' WORKFLOW")

process_document_response = workflow_operation.run_workflow(
    workflow_name="process_document", doc_id=document_id, data={}
)

pprint(process_document_response.model_dump())

print(f"RUN ID: {process_document_response.run_id}")

RUNNING 'process_document' WORKFLOW
{'created_at': '2024-10-18T03:28:34.000000+00:00',
 'document_id': '6711d5a67212c1ec5f894923',
 'document_name': 'AAPL_10Q.pdf',
 'end_date': None,
 'in_folders': [],
 'run_id': '6711d5a67212c1ec5f894923_354e1493-d922-4dcb-a928-fa4454dd9c06',
 'start_date': None,
 'state': None,
 'workflow_id': 'process_document'}
RUN ID: 6711d5a67212c1ec5f894923_354e1493-d922-4dcb-a928-fa4454dd9c06


#### The cell below utilizes the get workflow status API to check if the workflow has completed.

In [9]:
import time
import random

finished = False

while not finished:
    time.sleep(random.randint(5, 10))
    workflow_status_response = workflow_operation.get_workflow_status(
        show_internal_steps=False,
        workflow_id=process_document_response.workflow_id,
        workflow_run_id=process_document_response.run_id,
    )
    current_status = workflow_status_response.status
    print(f"Processing status: {current_status}")
    if current_status in ["success", "failed"]:
        finished = True

pprint(workflow_status_response.model_dump())

Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
Processing status: running
P

KeyboardInterrupt: 

##### Form processing can be run only after `process_document` workflow has finished

##### First we need to create a form definition to be able to run form extraction

#### Creating form definition

In [10]:
from weavaidev import forms as form_operation
from weavaidev.documents.models import CreateFormRequest

fields_data = [
    {
        "name": "Total Sales",
        "field_type": "Number",
        "description": "Net Sales for the quarter",
        "is_array": True,
        "fill_by_search": False,
    }
]
body = CreateFormRequest(
    name="APPLE FORM DEFINITION 2",
    description="Apple form definition 2",
    category=document_category,
    fields=fields_data,
    is_shared=True,
    is_searchable=True,
)
form_create_response = form_operation.create_form(form_data=body)

In [11]:
print(f"Form ID: {form_create_response.id}")

form_id = form_create_response.id

Form ID: 6711d89aec09d6e1f477c3f8


#### Process form workflow

In [12]:
print("RUNNING 'process_form_workflow' WORKFLOW")


process_form_response = workflow_operation.run_workflow(
    workflow_name="process_form_workflow", doc_id=document_id, data={"form_id": form_id}
)

pprint(process_form_response.model_dump())

print(f"FORM EXTRACTION RUN ID: {process_form_response.run_id}")

workflow_id = process_form_response.workflow_id
run_id = process_form_response.run_id

RUNNING 'process_form_workflow' WORKFLOW
{'created_at': '2024-10-18T03:40:29.000000+00:00',
 'document_id': '6711d5a67212c1ec5f894923',
 'document_name': 'AAPL_10Q.pdf',
 'end_date': None,
 'in_folders': [],
 'run_id': '6711d5a67212c1ec5f894923_a3bf03dc-45ab-44b9-9316-7b50d0541b85',
 'start_date': None,
 'state': None,
 'workflow_id': 'process_form_workflow'}
FORM EXTRACTION RUN ID: 6711d5a67212c1ec5f894923_a3bf03dc-45ab-44b9-9316-7b50d0541b85



### Get workflow status

In [25]:
import time
import random

finished = False

while not finished:
    time.sleep(random.randint(5, 10))
    workflow_status_response = workflow_operation.get_workflow_status(
        show_internal_steps=False,
        workflow_id=workflow_id,
        workflow_run_id=run_id,
    )
    current_status = workflow_status_response.status
    print(f"Processing status: {current_status}")
    if current_status in ["success", "failed"]:
        finished = True

pprint(workflow_status_response.model_dump())

Processing status: success
{'document_id': '66fe5c58b1d0dfb13c9975f3',
 'end_date': datetime.datetime(2024, 10, 3, 9, 26, 42, 419716, tzinfo=TzInfo(UTC)),
 'start_date': datetime.datetime(2024, 10, 3, 9, 25, 46, 190941, tzinfo=TzInfo(UTC)),
 'status': 'success',
 'tasks': [{'end_date': datetime.datetime(2024, 10, 3, 9, 26, 35, 407564, tzinfo=TzInfo(UTC)),
            'failed_task_ids': [],
            'name': 'extract_form_values_normal',
            'start_date': datetime.datetime(2024, 10, 3, 9, 26, 16, 324496, tzinfo=TzInfo(UTC)),
            'status': 'success',
            'task_status_summary': {'failed': 0,
                                    'queued': 0,
                                    'running': 0,
                                    'skipped': 0,
                                    'success': 1}},
           {'end_date': datetime.datetime(2024, 10, 3, 9, 25, 48, 124827, tzinfo=TzInfo(UTC)),
            'failed_task_ids': [],
            'name': 'set_processing_to_in_state

### Get single workflow

In [13]:
single_workflow_response = workflow_operation.get_single_workflow(
    show_internal_steps=False,
    workflow_name="process_form_workflow",
)
pprint(single_workflow_response.model_dump())

2024-10-17 20:40:39.028 | INFO     | weavaidev.workflows.service:get_single_workflow:58 - 200


{'name': 'process_form_workflow',
 'params': [],
 'tasks': [{'downstream_tasks': ['extract_form_values_list',
                                 'extract_form_values_normal'],
            'is_active': True,
            'name': 'get_document'},
           {'downstream_tasks': ['update_doc'],
            'is_active': True,
            'name': 'extract_by_search'},
           {'downstream_tasks': ['update_doc'],
            'is_active': True,
            'name': 'extract_form_values_list'},
           {'downstream_tasks': ['update_doc'],
            'is_active': True,
            'name': 'extract_form_values_normal'},
           {'downstream_tasks': ['set_processing_to_failed_state',
                                 'set_processing_to_in_state__1'],
            'is_active': True,
            'name': 'update_doc'},
           {'downstream_tasks': [],
            'is_active': True,
            'name': 'run_next_workflow'},
           {'downstream_tasks': ['run_next_workflow'],
            'is

### Rerun failed workflow (Optional)

In [ ]:
rerun_workflow = workflow_operation.rerun_workflow(
    workflow_name="process_form_workflow", data=running_workflow_data
)
pprint(rerun_workflow.model_dump())

{'created_at': '2024-10-03T04:01:26.000000+00:00',
 'document_id': '66fe11c5927ce8c0ebda42a3',
 'document_name': 'MCS-CS-Handbook-2022-2023Publish.pdf',
 'end_date': None,
 'in_folders': [],
 'run_id': '66fe11c5927ce8c0ebda42a3_c0d7d1fd-70be-4aba-a2bf-1bc1b960392c',
 'start_date': None,
 'state': None,
 'workflow_id': 'process_form_workflow'}


In [ ]:
workflow_status_response = workflow_operation.get_workflow_status(
    show_internal_steps=False,
    workflow_id=rerun_workflow.workflow_id,
    workflow_run_id=rerun_workflow.run_id,
)
pprint(workflow_status_response.model_dump())

{'document_id': '66fe11c5927ce8c0ebda42a3',
 'end_date': None,
 'start_date': datetime.datetime(2024, 10, 3, 3, 58, 9, 22120, tzinfo=TzInfo(UTC)),
 'status': 'running',
 'tasks': [{'end_date': datetime.datetime(2024, 10, 3, 3, 58, 58, 918824, tzinfo=TzInfo(UTC)),
            'failed_task_ids': [-1],
            'name': 'set_processing_to_in_state__1',
            'start_date': datetime.datetime(2024, 10, 3, 3, 58, 58, 918824, tzinfo=TzInfo(UTC)),
            'status': 'failed',
            'task_status_summary': {'failed': 1,
                                    'queued': 0,
                                    'running': 0,
                                    'skipped': 0,
                                    'success': 0}},
           {'end_date': None,
            'failed_task_ids': [],
            'name': 'extract_form_values_list',
            'start_date': None,
            'status': 'queued',
            'task_status_summary': {'failed': 0,
                                    'queu

### Get form

In [14]:
from weavaidev import forms as form_operation
from weavaidev.documents.models import FilterFormInstanceRequest


body = FilterFormInstanceRequest(
    scope="all_documents",
    doc_id=document_id,
)

form_create_response = form_operation.filter_form_instances(form_data=body).model_dump()

pprint(form_create_response)

{'form_instances': [], 'total': 0}


In [28]:
form_id = form_create_response["form_instances"][0]["form_id"]
form_id

'66fe234870dd6d497d9b8ba5'

### View form definition

In [15]:
form_definition_response = form_operation.get_form_definition(form_id=form_id)
pprint(form_definition_response.model_dump())

{'category': '',
 'created_at': '2024-10-18T03:40:10Z',
 'description': 'Apple form definition 2',
 'fields': [{'description': 'Net Sales for the quarter',
             'field_type': 'Number',
             'fill_by_search': False,
             'is_array': True,
             'name': 'Total Sales'}],
 'id': '6711d89aec09d6e1f477c3f8',
 'is_searchable': True,
 'is_shared': True,
 'name': 'APPLE FORM DEFINITION 2',
 'user_id': 'google-oauth2|117349365869611297391'}
